## Лабораторная работа №5

### N нитей M процессов, N:M = 3:5
Вариант 21
1. Реализовать 5  процессов, в каждом по три нити c фильтром и общим процесс-безопасным журналом, l (Log) c уровнями Info, Debug, прерываниями. Работа с журналом осуществляется в фоновом режиме. 
2. Сообщение в журнале формата 
     * начало/окончание, номер-нити/процесса  
     * главный процесс, окончание    

Фильтр, вставляющий символ ‘*’ после каждых 3-х символов входной строки.

In [1]:
# example of logging from multiple processes in a process-safe manner
from random import random
from time import sleep
from multiprocessing import current_process
from multiprocessing import Process
from multiprocessing import Queue
from threading import Thread
from logging.handlers import QueueHandler
import logging


def star(s):
    result = ""
    for i in range(len(s)):
        if (i % 3 == 2):
            result += "*"
        result += s[i]
    # sleep(random())
    print(result)

 
# executed in a process that performs logging
def logger_process(queue):
    # create a logger
    logger = logging.getLogger('app')
    # configure a stream handler
    logger.addHandler(logging.StreamHandler())
    # log all messages, debug and up
    logger.setLevel(logging.INFO)
    # run forever
    while True:
        # consume a log message, block until one arrives
        message = queue.get()
        # check for shutdown
        if message is None:
            break
        # log the message
        logger.handle(message)


# task to be executed in child processes
def threads(queue, strings):
    # create a logger
    logger = logging.getLogger('app')
    # add a handler that uses the shared queue
    logger.addHandler(QueueHandler(queue))
    # log all messages, debug and up
    logger.setLevel(logging.INFO)
    
    process = current_process()

    logger.info(f'Child {process.pid} starting.')
    

    threads = [Thread(target=star, args=((strings[i]),)) for i in range(3)]

    for thread in threads:
        logger.info(f'Child {thread.name} starting')
        thread.start()
        logger.info(f'Child {thread.name} has been ended')

    for thread in threads:
        thread.join()

    logger.info(f'Child {process.pid} done.')



# protect the entry point
if __name__ == '__main__':
    # create the shared queue
    queue = Queue()
    logger = logging.getLogger('app')
    # add a handler that uses the shared queue
    logger.addHandler(QueueHandler(queue))
    # log all messages, debug and up
    logger.setLevel(logging.INFO)
    # start the logger process
    logger_p = Process(target=logger_process, args=(queue,))
    logger_p.start()
    
    logger.info('Main process started.')

    article = "The eye is a region of mostly calm weather at the center of a tropical cyclone. It is roughly circular, and is typically 30 to 65 kilometers (19 to 40 miles) in diameter. The eye is surrounded by the eyewall, a ring of towering thunderstorms where the most severe weather and highest winds of the storm occur. In strong tropical cyclones, the eye is characterized by light winds and clear skies, surrounded on all sides by a towering, symmetric eyewall, while in weaker storms, it is less well defined and may be covered by a central dense overcast. Weaker or disorganized storms may also feature an eyewall that does not completely encircle the eye or have an eye that features heavy rain. In all storms, however, the eye is the location of the storm's minimum barometric pressure; this can be as much as 15 percent lower than the pressure outside the storm. This photograph, taken by the German astronaut Alexander Gerst from the International Space Station in September 2018, shows the well-defined eye of Hurricane Florence in the Atlantic Ocean. The Day of Reconciliation is a public holiday in South Africa held annually on 16 December. The holiday came into effect in 1995 after the end of apartheid, with the intention of fostering reconciliation and national unity for the country.[1] The date was chosen because it was significant to both Afrikaner and black African cultures. The government chose a meaningful date for both ethnic groups because they recognised the need for racial harmony. The celebration of the Day of Reconciliation can take the form of remembering past history, recognising veteran's contributions, marching, and other festivities. The eye is a region of mostly calm weather at the center of a tropical cyclone. The government chose a meaningful date for both ethnic groups because they recognised the need for racial harmony."
    arguments = tuple(article.split("."))


    processes = [Process(target=threads, args=(queue, arguments[i:i + 3],)) for i in range(5)]
    
    for process in processes:
        process.start()
        
    for process in processes:
        process.join()
        
    logger.info('Main process done.')
    
    queue.put(None)


Main process started.
Main process started.
Main process started.
Child 1160635 starting.
Child 1160635 starting.
Child Thread-4 (star) starting


 I*n s*tro*ng *tro*pic*al *cyc*lon*es,* th*e e*ye *is *cha*rac*ter*ize*d b*y l*igh*t w*ind*s a*nd *cle*ar *ski*es,* su*rro*und*ed *on *all* si*des* by* a *tow*eri*ng,* sy*mme*tri*c e*yew*all*, w*hil*e i*n w*eak*er *sto*rms*, i*t i*s l*ess* we*ll *def*ine*d a*nd *may* be* co*ver*ed *by *a c*ent*ral* de*nse* ov*erc*ast

Child Thread-4 (star) starting
Main process started.


Th*e e*ye *is *a r*egi*on *of *mos*tly* ca*lm *wea*the*r a*t t*he *cen*ter* of* a *tro*pic*al *cyc*lon*e

Child 1160635 starting.


...

 T*he *eye* is* su*rro*und*ed *by *the* ey*ewa*ll,* a *rin*g o*f t*owe*rin*g t*hun*der*sto*rms* wh*ere* th*e m*ost* se*ver*e w*eat*her* an*d h*igh*est* wi*nds* of* th*e s*tor*m o*ccu*r
 I*n s*tro*ng *tro*pic*al *cyc*lon*es,* th*e e*ye *is *cha*rac*ter*ize*d b*y l*igh*t w*ind*s a*nd *cle*ar *ski*es,* su*rro*und*ed *on *all* si*des* by* a *tow*eri*ng,* sy*mme*tri*c e*yew*all*, w*hil*e i*n w*eak*er *sto*rms*, i*t i*s l*ess* we*ll *def*ine*d a*nd *may* be* co*ver*ed *by *a c*ent*ral* de*nse* ov*erc*ast

 I*n s*tro*ng *tro*pic*al *cyc*lon*es,* th*e e*ye *is *cha*rac*ter*ize*d b*y l*igh*t w*ind*s a*nd *cle*ar *ski*es,* su*rro*und*ed *on *all* si*des* by* a *tow*eri*ng,* sy*mme*tri*c e*yew*all*, w*hil*e i*n w*eak*er *sto*rms*, i*t i*s l*ess* we*ll *def*ine*d a*nd *may* be* co*ver*ed *by *a c*ent*ral* de*nse* ov*erc*ast

Child Thread-4 (star) starting


Child Thread-4 (star) starting
Child Thread-4 (star) starting
Main process started.


Child 1160692 starting.
Child 1160692 starting.
Child Thread-4 (star) starting
Child Thread-4 (star) starting
Child Thread-4 (star) has been ended
Child Thread-4 (star) has been ended
Child Thread-5 (star) starting
Child Thread-5 (star) starting
Child 1160635 starting.
Child Thread-4 (star) has been ended
Child Thread-4 (star) has been ended
Child Thread-5 (star) starting


Child Thread-5 (star) starting
Child 1160659 starting.
Child 1160659 starting.
Child Thread-5 (star) has been ended
Child Thread-5 (star) has been ended
Child Thread-5 (star) has been ended
Child Thread-6 (star) starting
Child Thread-6 (star) starting
Child Thread-5 (star) has been ended
Child Thread-6 (star) starting
Child Thread-6 (star) starting
Child Thread-6 (star) has been ended
Child Thread-6 (star) has been ended
Child 1160714 starting.
Child 1160692 done.
Child 1160692 done.
Child 1160714 starting.
Child Thread-4 (star) starting
Child Thread-4 (star) starting
Child Thread-4 (star) starting
Child Thread-6 (star) has been ended
Child Thread-6 (star) has been ended
Child 1160635 starting.
Child Thread-4 (star) has been ended
Child Thread-4 (star) has been ended
Child Thread-5 (star) starting
Child Thread-5 (star) starting
Child Thread-4 (star) starting
Child Thread-4 (star) has been ended
Child Thread-4 (star) has been ended
Child Thread-4 (star) has been ended
Child Thread-5 (st